In [4]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr
import math
from scipy.stats import wasserstein_distance

In [5]:
Predicted = pd.read_csv("./Seurat/Outputs/output_Sim5.csv")
Target=pd.read_csv("./Data/Sim5/target.csv")

In [6]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
#Predicted.drop(columns="cell_ID", inplace=True)
Predicted.drop(columns="prediction.score.max",inplace=True)


In [7]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [8]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [9]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [10]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [11]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [12]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [13]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [14]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [15]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [16]:
np.mean(np.nan_to_num(d))

0.7274800247880797

In [17]:
d

array([0.69118684, 0.82438074, 0.48226445, 0.83184976, 0.85890984,
       0.6592176 , 0.73999972, 0.81657281, 0.7306164 , 0.83857596,
       0.67699917, 0.73931041, 0.56735663])

## Jenson Shannon Divergence

In [18]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.3985250439210602

## F1 Score, Precision, Recall

In [19]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [20]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [21]:
d

,Precision,Recall,F1-score
0.100,0.846847,0.541592,0.660663
0.200,0.803807,0.503901,0.619464
0.300,0.697103,0.513583,0.591434
0.010,0.741349,0.709457,0.725052
0.001,0.716683,0.743598,0.729893


## KL Divergence

In [22]:
scipy.stats.wasserstein_distance(np.reshape(Target,(Target.shape[0]*Target.shape[1])),np.reshape(Predicted,(Predicted.shape[0]*Predicted.shape[1])))

0.039061930429812264

In [23]:
to_be_kept=(Predicted>0)

In [24]:
Predicted=Predicted[to_be_kept]
Target=Target[to_be_kept]

In [25]:
kl=scipy.special.rel_entr(Target,Predicted)

In [26]:
sum(kl)/len(kl)

0.1304720804112447

In [27]:
len(Predicted)

3058

In [1179]:
np.reshape(Target,(Target.shape[0]*Target.shape[1])).shape

(8000,)